In [176]:
import numpy as np
import pandas as pd
import json
import sklearn
from sklearn.preprocessing import LabelEncoder 
from pandas.io.json import json_normalize
import seaborn as sns
sns.set_style("whitegrid")
from pandas.plotting import scatter_matrix

### Useful Sections:
Regression Notebook 3 "Why add Qualitative Data to Regression"

Regression Notebook 3 "How to Choose the Best Model" - k-fold cross validation

Unsupervised Learning Notebook 1 - k-Means clustering

# Cleaning

In [177]:
bot_labels = pd.read_csv('data/botwiki-2019.tsv', names = ['id', 'identity'], sep = '\t')
human_labels = pd.read_csv('data/verified-2019.tsv', names = ['id', 'identity'], sep = '\t')
bot_labels['bot'] = 1
human_labels['bot'] = 0
#IMPROVE?
# First set all beers as 0
#beers_train['Stout'] = 0
# Then locate all the stouts and set Stout to 1
#beers_train.loc[beers_train.Beer_Type == "Stout",'Stout'] = 1

In [178]:
#if IOUP data rate exceeded: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

#convert json to DataFrame
file = open('data/botwiki-2019_tweets.json')
file1 = open('data/verified-2019_tweets.json')
jsn = json.load(file)
jsn1 = json.load(file1)
bot_accts = pd.DataFrame(pd.json_normalize(jsn))
human_accts = pd.DataFrame(pd.json_normalize(jsn1))

In [179]:
#merge users and labels
bot_accts = bot_accts.merge(bot_labels, left_on=bot_accts['user.id'], right_on=bot_labels['id'])
human_accts = human_accts.merge(human_labels, left_on=human_accts['user.id'], right_on=human_labels['id'])
accts = bot_accts.append(human_accts)
#accts.to_csv('bot_wiki_verified_19.csv', index = False)

In [180]:
#filters accts to only numerical variables for simple knn

accts_knn = accts.filter(items = ['user.id', 'user.followers_count', 'user.friends_count', 'user.listed_count', 'user.favourites_count', 
                    'bot'])

accts_knn.tail()

,user.id,user.followers_count,user.friends_count,user.listed_count,user.favourites_count,bot
1982,76235237,3020,790,18,1707,0
1983,2159329092,4074,4239,163,4777,0
1984,25527618,17014,2610,608,7765,0
1985,43654274,367516,20245,5470,10282,0
1986,19084210,55512,11725,1366,2429,0


# Loading Data

In [222]:
accts = pd.read_csv('data/Cumulative_raw_data.csv')

In [240]:
pd.set_option('display.max_colwidth', None) # prevents descriptions from being cut off

44595


# Transmuting Text to Numbers

In [181]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [126]:
accts['user.description'].str.strip()


,key_0,created_at,user.id,user.id_str,user.name,user.screen_name,user.location,user.description,user.url,user.entities.url.urls,...,user.default_profile,user.default_profile_image,user.following,user.follow_request_sent,user.notifications,user.translator_type,id,identity,bot,desc_vectorized
0,969398693249126400,Wed Jun 05 21:43:20 +0000 2019,969398693249126400,969398693249126400,Random Goth Lyric Generator,goth_lyrics,City of Dis,Esoteric dark verse from aeons before twitter crept out of its unholy sepulchre.,https://t.co/s9UkVMDACm,"[{'url': 'https://t.co/s9UkVMDACm', 'expanded_url': 'https://osric.com/rglg/', 'display_url': 'osric.com/rglg/', 'indices': [0, 23]}]",...,False,False,False,False,False,none,969398693249126400,bot,1,"(0, 251)\t1\n (0, 626)\t1\n (0, 1404)\t1\n (0, 1817)\t1\n (0, 2159)\t1\n (0, 2784)\t1\n (0, 3792)\t1\n (0, 3880)\t1\n (0, 5490)\t1\n (0, 5622)\t1\n (1, 864)\t1\n (1, 1467)\t1\n (1, 1663)\t1\n (1, 2160)\t1\n (1, 2638)\t1\n (1, 2784)\t1\n (1, 3742)\t1\n (1, 3792)\t2\n (1, 4377)\t1\n (1, 5012)\t1\n (1, 5245)\t1\n (1, 5332)\t1\n (1, 5356)\t1\n (1, 5413)\t2\n (1, 5480)\t1\n :\t:\n (2682, 3792)\t1\n (2683, 1098)\t3\n (2683, 1407)\t1\n (2683, 2551)\t3\n (2683, 2707)\t1\n (2684, 87)\t1\n (2684, 146)\t1\n (2684, 507)\t1\n (2684, 1971)\t1\n (2684, 2275)\t1\n (2684, 2449)\t1\n (2684, 2459)\t1\n (2684, 3033)\t1\n (2684, 3048)\t1\n (2684, 3071)\t1\n (2684, 3792)\t2\n (2684, 3798)\t1\n (2684, 4275)\t1\n (2684, 4370)\t1\n (2684, 4473)\t1\n (2684, 4916)\t1\n (2684, 5245)\t1\n (2684, 5356)\t1\n (2684, 5757)\t1\n (2684, 5985)\t1"
1,2402022649,Tue Apr 02 13:26:02 +0000 2019,2402022649,2402022649,Transiting Now,transitingnow,Earth,Demonstrating the vast number of transiting exoplanets with real-time tweets when each transiting exoplanet appears to pass in front of its star - by @brettmor.,https://t.co/L8ARV5JWBz,"[{'url': 'https://t.co/L8ARV5JWBz', 'expanded_url': 'http://brettmorr.is', 'display_url': 'brettmorr.is', 'indices': [0, 23]}]",...,False,False,False,False,False,none,2402022649,bot,1,"(0, 251)\t1\n (0, 626)\t1\n (0, 1404)\t1\n (0, 1817)\t1\n (0, 2159)\t1\n (0, 2784)\t1\n (0, 3792)\t1\n (0, 3880)\t1\n (0, 5490)\t1\n (0, 5622)\t1\n (1, 864)\t1\n (1, 1467)\t1\n (1, 1663)\t1\n (1, 2160)\t1\n (1, 2638)\t1\n (1, 2784)\t1\n (1, 3742)\t1\n (1, 3792)\t2\n (1, 4377)\t1\n (1, 5012)\t1\n (1, 5245)\t1\n (1, 5332)\t1\n (1, 5356)\t1\n (1, 5413)\t2\n (1, 5480)\t1\n :\t:\n (2682, 3792)\t1\n (2683, 1098)\t3\n (2683, 1407)\t1\n (2683, 2551)\t3\n (2683, 2707)\t1\n (2684, 87)\t1\n (2684, 146)\t1\n (2684, 507)\t1\n (2684, 1971)\t1\n (2684, 2275)\t1\n (2684, 2449)\t1\n (2684, 2459)\t1\n (2684, 3033)\t1\n (2684, 3048)\t1\n (2684, 3071)\t1\n (2684, 3792)\t2\n (2684, 3798)\t1\n (2684, 4275)\t1\n (2684, 4370)\t1\n (2684, 4473)\t1\n (2684, 4916)\t1\n (2684, 5245)\t1\n (2684, 5356)\t1\n (2684, 5757)\t1\n (2684, 5985)\t1"
2,2976541239,Wed Jun 05 20:22:03 +0000 2019,2976541239,2976541239,TheRiddlerBot,TheRiddlerBot,"Coimbra, Portugal","Solve the riddle by replying only the name of the person/character described! Created by @verhoevenben, @1vangro, @fvancesco, @pedrojmmartins at #codecampcc",https://t.co/1v8BON9QpT,"[{'url': 'https://t.co/1v8BON9QpT', 'expanded_url': 'https://github.com/ivangro/theriddlerbot', 'display_url': 'github.com/ivangro/therid…', 'indices': [0, 23]}]",...,True,False,False,False,False,none,2976541239,bot,1,"(0, 251)\t1\n (0, 626)\t1\n (0, 1404)\t1\n (0, 1817)\t1\n (0, 2159)\t1\n (0, 2784)\t1\n (0, 3792)\t1\n (0, 3880)\t1\n (0, 5490)\t1\n (0, 5622)\t1\n (1, 864)\t1\n (1, 1467)\t1\n (1, 1663)\t1\n (1, 2160)\t1\n (1, 2638)\t1\n (1, 2784)\t1\n (1, 3742)\t1\n (1, 3792)\t2\n (1, 4377)\t1\n (1, 5012)\t1\n (1, 5245)\t1\n (1, 5332)\t1\n (1, 5356)\t1\n (1, 5413)\t2\n (1, 5480)\t1\n :\t:\n (2682, 3792)\t1\n (2683, 1098)\t3\n (2683, 1407)\t1\n (2683, 2551)\t3\n (2683, 2707)\t1\n (2684, 87)\t1\n (2684, 146)\t1\n (2684, 507)\t1\n (2684, 1971)\t1\n (2684, 2275)\t1\n (2684, 2449)\t1\n (2684, 2459)\t1\n (2684, 3033)\t1\n (2684, 3048)\t

In [149]:
text = open('sample.txt', 'w', encoding = 'utf-8')

In [150]:
vectorizer = CountVectorizer()
for desc in accts['user.description']:
       text.write(desc.strip().replace('\n+', ''))
text.close

<function TextIOWrapper.close()>

In [151]:
text = open('sample.txt', 'r', encoding = 'utf-8')

In [152]:
vectorizer.fit(text)
print(vectorizer.vocabulary_)


{'esoteric': 3909, 'dark': 3047, 'verse': 11396, 'from': 4503, 'aeons': 636, 'before': 1400, 'twitter': 11089, 'crept': 2859, 'out': 7948, 'of': 7775, 'its': 5756, 'unholy': 11209, 'sepulchre': 9603, 'demonstrating': 3196, 'the': 10583, 'vast': 11352, 'number': 7683, 'transiting': 10926, 'exoplanets': 4013, 'with': 11817, 'real': 8898, 'time': 10774, 'tweets': 11075, 'when': 11717, 'each': 3578, 'exoplanet': 4012, 'appears': 959, 'to': 10824, 'pass': 8085, 'in': 5507, 'front': 4504, 'star': 10098, 'by': 1908, 'brettmor': 1758, 'solve': 9928, 'riddle': 9158, 'replying': 9025, 'only': 7839, 'name': 7341, 'person': 8192, 'character': 2169, 'described': 3222, 'created': 2842, 'verhoevenben': 11386, '1vangro': 123, 'fvancesco': 4551, 'pedrojmmartins': 8147, 'at': 1113, 'codecampcc': 2401, 'suisea': 10268, 'donate': 3442, 'https': 5330, 'co': 2379, 'ms5qm3a8bl': 7237, 'e70fqkzkdj': 3574, 'q6hc8oxlrg': 8747, 'inspired': 5613, 'tiny_star_field': 10792, 'make': 6707, 'little': 6504, 'word': 118

In [166]:
for i in range(len(accts['user.description'])):
    text = accts['user.description'][i]
    accts['user.description'][i] = vectorizer.transform(text)
accts['desc_vectorized'] = vectorizer.transform(accts['user.description'])
accts['desc_vectorized'] = accts['desc_vectorized'].toarray()

AttributeError: lower not found

### TfidfVectorizer

In [241]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [242]:
accts_copy = accts.copy()
accts_train = accts_copy.sample(frac = 0.75, random_state = 1400)
accts_test = accts_copy.drop(accts_train.index)

In [243]:
vectorizer = TfidfVectorizer()

In [228]:
#print(accts['user.description'])
vectorizer.fit(accts_train['description'].tolist())

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [196]:
print(vectorizer.vocabulary_)

{'dynasty': 2802, 'airs': 571, 'friday': 3556, 'nights': 5995, 'at': 894, 'pm': 6623, 'on': 6198, 'the': 8397, 'cw': 2337, 'diy': 2663, 'shows': 7690, 'round': 7323, 'world': 9455, 'doors': 2716, 'eight': 2888, 'of': 6160, 'gooffkings': 3821, 'blockedpartypod': 1282, 'mcdonald': 5417, 'grilled': 3896, 'chicken': 1794, 'mcwrap': 5421, '250': 154, 'calories': 1572, 'is': 4517, 'both': 1355, 'dollar': 2699, 'and': 696, 'pound': 6713, 'bargain': 1056, 'call': 1568, 'me': 5426, '802': 313, '359': 182, '7477': 290, 'ـنغـرد': 9721, 'ـل': 9720, 'كـلمة': 9724, 'في': 9722, 'اللـغة': 9691, 'العربيـة': 9690, 'تنتهي': 9700, 'المهمة': 9695, 'عام': 9715, '2019': 130, 'من': 9734, 'abdullaharif': 389, 'go': 3784, 'sports': 7957, 'generating': 3685, 'alignment': 609, 'charts': 1760, 'for': 3442, 'your': 9593, 'pleasure': 6611, 'by': 1543, 'astralborn': 887, 'washingtonpost': 9195, 'scribe': 7521, 'covering': 2207, 'military': 5560, 'state': 8014, 'school': 7486, 'kid': 4780, 'via': 9059, 'umassamherst':

In [197]:
print(vectorizer.idf_)

[7.22208011 6.52893293 7.91522729 ... 7.91522729 7.91522729 7.91522729]


In [198]:
accts_train['tfidf']= vectorizer.transform(accts_train['user.description'])

In [214]:
print(accts_train['user.description'][0])

80                                                                                                   editorial board @nytimes, lead writer on technology.
80    I'm a #bot that challenges people to ride the @BikeMetro bike share around the Port of LA. Made with ❤️ by @thzinc; not affiliated with @BikeMetro.
Name: user.description, dtype: object


In [213]:
print(accts_train['tfidf'][50])

  (0, 8397)	0.11725716940749849
  (0, 6623)	0.3393832502084331
  (0, 6198)	0.17016214957418085
  (0, 5995)	0.40418214829735705
  (0, 3556)	0.36687634425232685
  (0, 2802)	0.4260046447229513
  (0, 2337)	0.38869884067792115
  (0, 894)	0.17351208656217823
  (0, 571)	0.4260046447229513
  (1, 9455)	0.25588309178793706
  (1, 8397)	0.1230414982162103
  (1, 7690)	0.35612512041811384
  (1, 7323)	0.4241205661883651
  (1, 2888)	0.4470195724375412
  (1, 2716)	0.4470195724375412
  (1, 2663)	0.4241205661883651
  (1, 894)	0.18207148609427304
  (2, 6713)	0.24430166205912385
  (2, 6160)	0.14033100711496474
  (2, 5426)	0.12449809205092047
  (2, 5421)	0.24430166205912385
  (2, 5417)	0.24430166205912385
  (2, 4517)	0.1127350323048341
  (2, 3896)	0.24430166205912385
  (2, 3821)	0.24430166205912385
  :	:
  (2012, 7341)	0.2252694328750397
  (2012, 7140)	0.15938444845394134
  (2012, 6852)	0.22392150340264966
  (2012, 6346)	0.34672060630694296
  (2012, 6160)	0.09958109056742576
  (2012, 3812)	0.316357812948967

# K Nearest Neighbors

In [78]:
#K-fold cross validation
from sklearn.model_selection import KFold
from sklearn.base import clone

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [79]:
#Creating test train split for knn
X = accts_knn[['user.followers_count', 'user.friends_count', 'user.listed_count', 'user.favourites_count', ]].to_numpy()
y = accts_knn[['bot']].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.25, random_state=1400)
accts_knn_train = accts_knn.sample(frac = 0.75,  random_state = 876)

In [80]:
knn = KNeighborsClassifier(n_neighbors = 5)

In [81]:
knn.fit(X_train, y_train.ravel())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [82]:
y_predict = knn.predict(X_train)

In [83]:
print("Our model has a ",
      np.round(sum(y_predict == y_train.ravel())/len(y_train)*100,2),
      "% accuracy on the training set")

Our model has a  98.06 % accuracy on the training set


# K-means Clustering

In [27]:
from sklearn.cluster import KMeans

In [28]:
kmeans = KMeans(2)

In [29]:
kmeans.fit(accts_train)

ValueError: could not convert string to float: 'bot'

In [ ]:
#Test Train Split
